In [ ]:
import pandas as pd

# Primero se carga y lee el archivo original que la empresa nos facilitó.
archivo = "Base de datos ONIX SAS Original.xlsx"
bd = pd.ExcelFile(archivo)

# Acá verificamos que el archivo se haya leído bien y verificar las hojas.
print("Hojas del archivo:", bd.sheet_names)

# Para no repetir el mismo comando hoja por hoja, analizamos todas las hojas y verificamos los duplicados que haya.
print("\n===========================")
print("Cantidad de duplicados en cada hoja")
print("===========================")

for hoja in bd.sheet_names:
# Se lee cada hoja individualmente
    df_temp = bd.parse(hoja)
    
# Se cuentan cuántas filas están repetidas completamente
    duplicados = df_temp.duplicated().sum()
    
# Se muestra el total para cada hoja
    print(f"Hoja '{hoja}': {duplicados} duplicados")


# La hoja de ventas necesita revisiones adicionales porque es la más importante, con más datos y la más propensa errores.

df = bd.parse("Ventas")

# Aquí revisamos qué columnas tienen información faltante y saber si hay que reconstruir valores, eliminar filas o aplicar reglas del negocio.
print("\n===========================")
print("Valores nulos en la hoja de ventas")
print("===========================")
print(df.isna().sum().to_string())


# Convertimos la columna Fecha a tipo datetime para poder extraer el año y así identificar cuales están mal registradas.
df["Fecha"] = pd.to_datetime(df["Fecha"], errors="coerce")

# Filtramos las fechas que no correspondan a 2024 ya que la base de datos registra toda la operación de ese año.
fechas_incorrectas = df[df["Fecha"].dt.year != 2024]

print("\n===============================")
print("Cantidad de fechas con año distinto a 2024")
print("===============================")

# Mostramos el total de errores detectados
print("Cantidad de fechas incorrectas:", len(fechas_incorrectas))

In [1]:
#Se importan las librerias necesarias para conectar VSCode con Excel y manipular y managear datos como las fehcas y caracteres especiales
import pandas as pd
import numpy as np
import unicodedata
from datetime import datetime

# Para evitar hacer el código extremadamente largo al repetir varias veces los commando que realizan la misma función, se crean funciones para cada tarea específica.

# En esta función es donde se hace uso del unicodedata para eliminar las tildes y armonizar el texto, en especial en la hoja de países.
def quitar_tildes(texto):
    if pd.isna(texto):
        return texto
    if not isinstance(texto, str):
        texto = str(texto)
    nfkd = unicodedata.normalize('NFKD', texto)
    return "".join([c for c in nfkd if not unicodedata.combining(c)])

# La base de dtaos viene con todos los nombres en mayúsculas, algo que se vería feo en el Dashboard, por lo que se creó una función que deje en mayuscula la primera letra de cada palabra.
def formato_nombre(texto):
    if pd.isna(texto):
        return texto
# Dentro de esta nueva función llamamos a la función quitar_tildes para que el armonizado del texto sea completo.    
    texto = quitar_tildes(str(texto)).strip().lower()
    palabras = texto.split()
    return " ".join([p.capitalize() for p in palabras])

# En la hoja de ventas,hay varias fechas cuyo año se registró mal, como es información de todo el 2024, creamos una función que corrija la fecha automaticamente a 2024.
def corregir_fecha(fecha):
    if pd.isna(fecha):
        return fecha
    if fecha.year != 2024:
        return fecha.replace(year=2024)
    return fecha

# Algunos descuentos se registraron como números enteros por error, lo que da descuentos super locos de 200%, por lo que para esos donde se supera el 100% (1) se divida por 100 y se corrija.
def corregir_descuento(desc):
    if pd.isna(desc):
        return 0
    if desc > 1:
        return desc / 100
    return desc

# En la columna de resumen venta es donde se indica si fue una venta normal o una muy grande, por lo que solo hay 2 valores posibles, esta función corrige los que eastán mal escritos y los que están vacíos.
def limpiar_resumen(texto):
# Si está vacio automaticamente se toma como vendido    
    if pd.isna(texto) or str(texto).strip() == "":
        return "Vendido"

    texto = quitar_tildes(str(texto)).strip().lower()

# Si contiene la palabra gran se toma como gran venta    
    if "gran" in texto:
        return "Gran Venta"

# Si hay algo difefrente a gran venta o vacio, automaticamente se toma como vendido también.
    return "Vendido"

# Aquí ya cargamos el archivo original de la empresa que nos facilitaron

# Creamos una variable para nuestra base de datos y evitar tener que escribir el nombre del archivo cada vez, la abrimos con pandas e indicamos que se debe leer la información de cada una de las hojas.
archivo = "Base de datos ONIX SAS Original.xlsx"
bd = pd.ExcelFile(archivo)
hojas = {name: bd.parse(name) for name in bd.sheet_names}

# primero se ejecuta la limpieza de la hoja la hoja ventas que es la principal y la que más erores tiene.

df = hojas["Ventas"].copy()

# Se llama a la columna fechas de la hoja y se ajecuta la función de corrección de fechas.
df["Fecha"] = df["Fecha"].apply(corregir_fecha)

# Acá se llama a la columna de descuentos y se ejecuta la función que dicide sobre 100 para corregir los errores.
df["Descuento"] = df["Descuento"].apply(corregir_descuento)

# Para las columnas de valores faltantes tenemos la ventaja de que se pueden calcular con columnas ya establecidas y no es necesario promediar o eliminar la venta.

# Indicamos que todos los valores brutos se calculan con el precio unitario por las unidades que se vendieron.
df["Valor Bruto"] = df["Valor Bruto"].fillna(df["Precio Unitario"] * df["Unidades Vendidas"])

# Indicamos que el IVA precio siempre sera el valor bruto por la tasa de 19% de Colombia.
df["IVA"] = df["IVA"].fillna(df["Valor Bruto"] * 0.19)

# Para el costos total se toma el costo unitario y se multiplica por las unidades vendidas.
df["Costo Total"] = df["Costo Total"].fillna(df["Unidades Vendidas"] * df["Costo Unitario"])

# Para el total de la venta solo se indica que se debe sumar el IVA y el valor bruto.
df["Total Venta"] = df["Total Venta"].fillna(df["Valor Bruto"] + df["IVA"])

# Por último, para las rentabilidades vacias, hay que tomar el total de la venta, si hay se le resta el descuento y se le resta el total del costo.
df["Rentabilidad"] = df["Rentabilidad"].fillna((df["Total Venta"] * (1 - df["Descuento"])) - df["Costo Total"])

# Ejecutamos la función de limpieza de la columna de resumen venta.
df["Resumen Venta"] = df["Resumen Venta"].apply(limpiar_resumen)

#Quitamos todos las ventas duplicadas que haya en la hoja.
df = df.drop_duplicates()

# Limpiamos la hoja de ciudades aplicandole el formato de texto y eliminando las que estén duplicadas.
ciudades = hojas["Ciudades"].copy()
ciudades["Ciudad"] = ciudades["Ciudad"].apply(formato_nombre)
ciudades = ciudades.drop_duplicates()

# Limpiamos de igual forma la de vendores aplicandole el formato de texto y eliminando dulicados.
vendedores = hojas["Vendedores"].copy()
if "Vendedor" in vendedores.columns:
    vendedores["Vendedor"] = vendedores["Vendedor"].apply(formato_nombre)
vendedores = vendedores.drop_duplicates()

# Misma armonización de texto y eliminación de duplicados para los clientes.
clientes = hojas["Clientes"].copy()
if "Cliente" in clientes.columns:
    clientes["Cliente"] = clientes["Cliente"].apply(formato_nombre)
clientes = clientes.drop_duplicates()

# Finalmente, limpiamos aplicamos el formato armonizado y eliminas duplicados en la hoja de artículos.
art = hojas["Articulos"].copy()
art["Articulo"] = art["Articulo"].apply(formato_nombre)
art = art.drop_duplicates()


# Para finalizar, creamos un nuevo excel y guardamos cada una de las hojas ya limpias.
with pd.ExcelWriter("Base de datos ONIX Limpia.xlsx", engine="openpyxl") as writer:
    df.to_excel(writer, sheet_name="Ventas", index=False)
    art.to_excel(writer, sheet_name="Articulos", index=False)
    vendedores.to_excel(writer, sheet_name="Vendedores", index=False)
    clientes.to_excel(writer, sheet_name="Clientes", index=False)
    ciudades.to_excel(writer, sheet_name="Ciudades", index=False)

# Colocamos un print para corroborar que el proceso se realizó exitosamente.
print("base de datos limpiada y guardada en exitosamente en 'Base de datos ONIX Limpia.xlsx'")

base de datos limpiada y guardada en exitosamente en 'Base de datos ONIX Limpia.xlsx'
